In [1]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import uuid
import difflib
import html5lib
pd.options.mode.chained_assignment = None 

In [2]:
player_index = pd.read_csv('../updated_datasets/player_index.csv')
player_index = player_index[player_index.columns[1:]]
player_index

,player,uuid
0,Hot Rod Williams,ca94dd3e-c3dc-4682-8c88-0ed654c9d074
1,Danny Ferry,4d2819b0-30c9-40f3-af49-4b950b0fad78
2,Mark Price,1a806ca8-ea9c-4690-8fe4-0b7d13c80d36
3,Brad Daugherty,ae75e1f9-765b-4cde-afbc-b5e9f18e0a26
4,Larry Nance,97054146-250e-4988-aff6-40ce7e6f6c4d
...,...,...
2819,Vit Krejci,9b06ee3b-d2b2-4e58-9689-a11051808fde
2820,Olivier Sarr,d09f39f5-e68c-4f3d-ba3c-1e7946491fcd
2821,Lindy Waters III,a2d20bf5-1dab-4cfc-80e6-e0c848d31e9e
2822,Rob Edwards,f388acbe-7951-43e5-8778-dab1019c1843


In [12]:
null_players = pd.read_csv('../updated_datasets/null_players.csv')
null_players = null_players[null_players.columns[1:]]
null_players.columns = null_players.columns.str.lower()
null_players.head()

,new_name,original_name
0,Darrell Armstrong,Darrel Armstrong
1,Predrag Stojakovic,Peja Stojakovic
2,PJ Brown,P.J. Brown
3,Jaren Jackson Jr,Jaren Jackson
4,AJ Guyton,A.J. Guyton


In [53]:
def update_wins(table):
    table['adjusted_wins'] = pd.to_numeric(table['wins']) + abs(min(pd.to_numeric(table['wins']))) + 1
    return table 

In [54]:
def just_name(row):
    return str(row['name']).split(', ')[0].split('  ')[-1]

def just_pos(row):
    return str(row['temp']).split(', ')[-1]

def clean_table(table):
    table.columns = table.iloc[0].str.lower()
    table = table.tail(-1)
    table['temp'] = table['name']
    table['name'] = table.apply(just_name, axis=1)
    table['temp'] = table.apply(just_pos, axis=1)
    table.drop(columns=['rk'], inplace=True)
    table.rename(columns={'temp': 'pos', 'name': 'player'}, inplace=True)
    cols = table.columns.values
    cols = [cols[i] for i in [0, 8, 1, 2, 3, 4, 5, 6, 7]]
    table = table[cols]
    table = pd.merge(left=table, right=player_index, how='left', on='player')
    table = table[table['player'] != 'NAME']
    
    for bad_name in table['player']:
            if bad_name in [x for x in null_players['original_name']]:
                table.loc[table[table['player'] == bad_name].index, 'player'] = [
                    x for x in null_players[null_players['original_name'] == bad_name]['new_name']][0]
    
    #table.dropna(inplace=True)
    table.reset_index(drop=True)
    
    table = update_wins(table)
    return table

In [55]:
df_list = []
table_list = []
def create_tables():
    for year in range(1997, 2023):
        var_name = f'stats_{year - 1}'
        table_list.append(var_name)

        table_soup = soup(requests.get(f'http://www.espn.com/nba/statistics/rpm/_/year/{year}/page/1').text, 'html.parser')
        page_count = int(str(table_soup.select('div.page-numbers')).split('of ')[1].split('<')[0])
        table = pd.read_html(str(table_soup.select('table.tablehead')))[0]
        for i in range(2, page_count + 1):
            table_soup = soup(requests.get(f'http://www.espn.com/nba/statistics/rpm/_/year/{year}/page/{i}').text, 'html.parser')
            table = pd.concat([table, pd.read_html(str(table_soup.select('table.tablehead')))[0]])

        globals()[var_name] = clean_table(table)
        globals()[var_name].to_csv(f'../updated_datasets/stats_data/{var_name}.csv')
        df_list.append(globals()[var_name])
        print(f'{var_name} is done!')

    return table_list

In [56]:
create_tables()

stats_1996 is done!
stats_1997 is done!
stats_1998 is done!
stats_1999 is done!
stats_2000 is done!
stats_2001 is done!
stats_2002 is done!
stats_2003 is done!
stats_2004 is done!
stats_2005 is done!
stats_2006 is done!
stats_2007 is done!
stats_2008 is done!
stats_2009 is done!
stats_2010 is done!
stats_2011 is done!
stats_2012 is done!
stats_2013 is done!
stats_2014 is done!
stats_2015 is done!
stats_2016 is done!
stats_2017 is done!
stats_2018 is done!
stats_2019 is done!
stats_2020 is done!
stats_2021 is done!


['stats_1996',
 'stats_1997',
 'stats_1998',
 'stats_1999',
 'stats_2000',
 'stats_2001',
 'stats_2002',
 'stats_2003',
 'stats_2004',
 'stats_2005',
 'stats_2006',
 'stats_2007',
 'stats_2008',
 'stats_2009',
 'stats_2010',
 'stats_2011',
 'stats_2012',
 'stats_2013',
 'stats_2014',
 'stats_2015',
 'stats_2016',
 'stats_2017',
 'stats_2018',
 'stats_2019',
 'stats_2020',
 'stats_2021']

In [51]:
stats_1996

,player,pos,team,gp,mpg,orpm,drpm,rpm,wins,uuid,adjusted_wins
0,Michael Jordan,G,CHI,82,37.9,5.15,2.47,7.62,20.72,6cbad2dc-2d47-4b93-b998-97f104cef79c,24.86
1,Gary Payton,PG,SEA,82,39.2,3.66,2.52,6.18,19.22,901e08ca-0383-4677-9716-e0d94aaf1fdc,23.36
2,Shaquille O'Neal,C,LAL,51,38.1,3.42,2.63,6.04,11.80,4780ef7e-7c65-4ab0-b2ab-239f446cf97e,15.94
3,Terry Mills,F,DET,79,25.3,1.48,4.44,5.92,10.98,ebfc0d0c-20a2-4b02-bed3-2da068d81aa1,15.12
4,Tim Hardaway,G,MIA,81,38.7,4.25,1.58,5.83,18.58,96e17ae1-a423-4d71-890b-525d2395dcc3,22.72
...,...,...,...,...,...,...,...,...,...,...,...
438,Kevin Duckworth,C,LAC,26,14.8,-4.55,-2.16,-6.71,-0.91,af5f0718-d3bf-453d-be32-650509490dab,3.23
439,James Scott,G,MIA,8,4.0,-6.02,-1.06,-7.08,-0.07,9fffcb6f-ea78-48d3-bcb8-d56465be20a5,4.07
440,Lasalle Thompson,C,DEN/IND,26,5.4,-7.49,0.10,-7.38,-0.39,NaN,3.75
441,Tim Kempton,F,SA,10,5.9,-6.75,-0.97,-7.73,-0.17,ecbf877a-db1f-4383-b085-8dcd4b9bd8bc,3.97
